## NLP 파인튜닝 해보기

### 기본 구조
- 참고자료: https://huggingface.co/learn/nlp-course/en/chapter3/1?fw=pt

In [1]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# This is new
batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

- 허브는 모델 뿐만 아니라 데이터셋을 보유하고 있다.
- MRPC데이터셋을 이용해서 파인튜닝을 해보겠다. 
- 이 데이터셋은 5,801개의 문장 쌍으로 이루어져 있다. 문장들은 서로 비슷한 의미를 갖은 문장인지 아닌지에 대한 레이블이 있다.
- 데이터셋은 기본적으로 `~/.cache/huggingface/datasets`에 다운로드가 된다.
- 위치를 변경하고 싶으면 `HF_HOME`이라는 환경변수명을 이용해서 설정해보자.

In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Found cached dataset glue (C:/Users/user/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

- 문장 하나 확인해보기

In [3]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

- `.featrues`: 데이터셋의 피처 타입 알아보기
- 레이블의 타입이 `ClassLabel`이다. 정수를 매핑하고 있는 레이블이다. names폴더에 있다.

In [4]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

### 데이터셋 전처리

- 모델에 맞는 토크나이저를 이용해서 문장1, 문장2를 토큰화해보겠다.

In [5]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

- 토큰화된 문장들은 모델로 바로 학습시킬 수 없다. 두 문장을 한 쌍으로 묶여서 처리를 해야한다. 그래서 이 두 문장이 서로 관련이 있는지 없는지를 모델에게 전달해야 한다.
- 운이 좋게도, BERT의 토크나이저는 이 기능을 제공한다. 이 모델은 두 문장을 비교하는 사전학습을 진행했기 때문에 분류하는 기능이 있다. 다른 모델은 이 기능이 없을 수도 있다.
- 아래는 예를 든 문장. (본문과 관련이 없다.)

In [8]:
# 예시
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

# 출력문
'''
{
    'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 
    'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 
    'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
}
'''

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

- `token_type_is`는 문장1과 문장2를 분류해준다. 리스트의 0은 문장1, 뒤에 1은 문장2.
- 디코딩을 해서 다시 확인해보자.

In [10]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

- 이제 우리 데이터셋을 다시 학습시켜보자. 
- 예시에서 본 것처럼 두 문장을 그냥 제공해주면 안된다.
- 원리를 자세히 이해하지 못하겠지만, 토큰화된 딕셔너리를 돌려줄 때 아주 많은 데이터량 때문에 문제가 된다.
- 한번 아주 많은 양의 데이터셋을 모아두려고 하기 때문인 것 같다.

In [12]:
# tokenized_dataset = tokenizer(
#     raw_datasets["train"]["sentence1"],
#     raw_datasets["train"]["sentence2"],
#     padding=True,
#     truncation=True,
# )

# tokenized_dataset

- `Dataset.map()`을 이용해서 데이터셋의 요소들 하나하나씩 차례대로 토큰화한다.
- `batched=True`는 여러 문장을 한번에 다루게 한다. 문장 쌍 하나만을 다루지 않는다. 그래서 속도를 높힌다.
- `padding`옵션이 빠졌다. 모든 문장들을 전체 데이터셋의 맥시멈 크기에 맞게 패딩을 주는 건 매우 비효율적이다. 배치단위로 가장 긴 문장을 기준으로 패딩을 줘야 효율적이다. `dynamic padding`
- `num_proc` 멀티프로세싱을 지원한다. 이번 경우에는 사용하지 않았다. 토크나이저 함수 자체가 멀티프로세싱으로 작업을 처리한다.

In [13]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

- `collate function` 배치에 데이터를 모아두는 작업을 한다. 데이터를 파이토치의 텐서로 변환하고 concat한다. 그리고 `Data Loader`를 빌드할 때 전달한다.
- 이번 작업에서는 디폴트 콜래이트 함수를 사용할 수 없다. 왜냐하면 각 배치마다 인풋의 사이즈가 다르기 때문이다.
- 참고로 `TPU`를 사용하는 경우에는 배치 사이즈를 서로 다르게 하면 안된다. 고정된 사이즈를 선호한다.
- `DataCollateWithPadding`을 이용하면 토큰을 가져가서 어떤 패딩토큰을 이용해야 하는지, 패딩이 오른쪽이나 왼쪽 어느 곳에 있어야 하는지를 파악한다.

In [14]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

- `samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}`
  - k: v 형태의 딕셔너리를 만든다
  - 데이터셋에서 8개를 추출한 샘플에서
  - k가 idx, sen1, sen2가 아니라면
- 아래 코드는 인덱스, 문장1, 문장2를 배제하고 각 샘플의 아이디 리스트 길이를 모은 리스트다. 패딩 관련 설정을 하기 위해서다.


In [29]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

- 다이나믹 패딩 적용
- 모든 아이디리스트의 길이를 67로 고정

In [33]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}

### Trainer API를 이용해서 파인튜닝 해보기

- 트랜스포머의 `Trainer` 클래스는 사전학습 모델을 파인튜닝할 수 있는 도움을 준다.
- CPU 환경은 너무 느리고, GPU 환경을 권장한다.

In [39]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Found cached dataset glue (C:/Users/user/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-3dfcda83adde2396.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-11e216b6f01e882d.arrow


#### Training

- `TrainingArguments` 클래스는 `Trainer`가 훈련하고 평가하는데 사용하는데 모든 하이퍼파라미터를 가지고 있다. 
- 트레인된 모델이 어느 곳에 저장이 될지 디렉토리를 제공해주기만 하면 된다.
- 특별하게 파라미터를 설정하지 않아도 된다. 기본적인 파인튜닝만으로도 잘 작동한다.

In [40]:
from transformers import TrainingArguments

training_args = TrainingArguments("C:/data/huggingface/test1/test-trainer")

- checkpoint에 해당하는 아키텍처 가져오기
- sequence classification에 맞는 헤드가 기존에 있던 헤드를 대체한다.
- `Some weights of the model checkpoiint were not used`: 분리된 기존의 헤드
- `that some others were randomly initialized`: 새로 대체된 헤드

In [41]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

- ```
    trainer = Trainer(
        모델,
        하이퍼파라미터,
        토큰화된 트레인 데이터셋,
        토큰화된 밸리데이션 데이터셋,
        데이터셋 텐서화로 감싸고 배치에 올리는 콜래이터,
        토크나이저
    )
    ```
- data_collator=data_collator는 스킵해도 되는 라인이다. 트레이너에 의해 사용될 콜래이터는 이미 `DataCollaterWithPadding`로 정의했기 때문이다.


In [42]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

#### Evaluation

- `compute_metric()` 함수는 `EvalPrediction` 오브젝트를 이용할 것이다. 이 오브젝트는 `predictons` 필드와 `label_ids` 필드를 가지고 있는 이름이 달린 튜플이다.
- `predict()`에 의해 `metrics`가 생겨났다. 이 메트릭은 전달한 데이터셋에 대한 손신율과 프리딕트하는데 얼마나 오래 걸렸는지 그리고 그 평균에 대한 데이터를 가지고 있다. 
- 예측된 프리딕션은 2차원 어레이를 가지고 있다. (408x2) 408개의 문장에 대한 예측 로짓 2개.

In [44]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/51 [00:00<?, ?it/s]

(408, 2) (408,)


- 두 로짓 중 더 높은 값으로 어레이를 만듦

In [53]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1) # 열 비교

- 이제 레이블과 preds를 비교.
- `compute_metirc()` 함수를 만들기 위해서 `evalute` 라이브러리를 이용한다.
- 모델의 헤드가 랜덤하게 초기화됐기 때문에 결과는 서로 다르다.

In [56]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.6838235294117647, 'f1': 0.8122270742358079}

- 위에서 한 과정을 함수 하나로 래핑하기

In [57]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

- 각 에포크마다 메트릭을 리포트하기 위해 `training_args`와 `Trainer()`를 다시 설정한다.

In [58]:
training_args = TrainingArguments("C:/data/huggingface/test1/test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i